In [88]:
import yaml
import json
import os
import random
import shutil


folder = '/scratch/bioinf/projects/diploid-genome-assembly/pebert/test_clr/run_folder/input/fastq/HG00733_1kg_il25k-npe_sseq'

out_folder = '/scratch/bioinf/projects/diploid-genome-assembly/pebert/subsampling'
os.makedirs(out_folder, exist_ok=True)

all_files = os.listdir(folder)

all_libs = set([x.split('_')[3] for x in all_files if x.endswith('.fastq.gz')])

total_libs = len(all_libs)

all_subsets = []

while 1:

    for factor in [0.8, 0.6, 0.4, 0.2]:
        select_num = int(round(total_libs * factor))
        selected_libs = set(random.sample(all_libs, select_num))
        all_subsets.append(selected_libs)

    all_jaccards = []

    for i in all_subsets:
        for j in all_subsets:
            if i == j:
                continue
            isect = i.intersection(j)
            union = i.union(j)
            jaccard = len(isect) / len(union)
            all_jaccards.append(jaccard)

    if all([j < 0.5 for j in all_jaccards]):
        print('copying')
        for subset, label in zip(all_subsets, ['sub80', 'sub60', 'sub40', 'sub20']):
            out_path = os.path.join(out_folder, label)
            os.makedirs(out_path, exist_ok=True)
            for lib in subset:
                subset_files = [f for f in all_files if lib in f]
                for sf in subset_files:
                    new_file = sf.replace('1kg', label)
                    old_path = os.path.join(folder, sf)
                    new_path = os.path.join(out_path, new_file)
                    shutil.copy(old_path, new_path)
        break
    else:
        print('max j ', max(all_jaccards))
                
    
    





copying
